In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0.1
)


### from_template()메소드
* 변수를 중괄호로 묶어서 템플릿에 정의

In [3]:
from langchain_core.prompts import PromptTemplate

#template정의, {}안의 내용은 이후에 값이 들어갈 자리
template = "{name}의 직업은 무엇인가요"

#from_template메소드를 사용해 prompttemplate객체를 생성
prompt = PromptTemplate.from_template(template)

print(prompt)

input_variables=['name'] input_types={} partial_variables={} template='{name}의 직업은 무엇인가요'


In [4]:
#prompt생성(완성) format() 메소드를 이용해 변수값을 넣음
prompt = prompt.format(name="bear")
print(prompt)

bear의 직업은 무엇인가요


In [12]:
from langchain_core.prompts import PromptTemplate

template = "{language}는 누가 만들었나요"

prompt = PromptTemplate.from_template(template)

print(prompt)

chain = prompt | llm

input_variables=['language'] input_types={} partial_variables={} template='{language}는 누가 만들었나요'


In [9]:
#기본적으로 딕셔너리 형태로 변수를 작성해야하지만
#변수가 한개일때는 변수만으로 작성이 가능

# print(chain.invoke({"language":"Python"}).content)
print(chain.invoke("Python").content)

Python은 귀도 반 로섬(Guido van Rossum)에 의해 1991년에 처음 개발되었습니다. 그는 이 프로그래밍 언어를 만들 때 코드의 가독성과 사용의 용이성을 중시하였으며, Python은 이후로도 많은 개발자들 사이에서 인기를 끌게 되었습니다. 귀도 반 로섬은 Python의 "주요 개발자"로 오랫동안 활동하였고, 2018년에는 Python의 "자유로운 리더" 역할을 공식적으로 종료했습니다.


### prompttemplate객체 생성과 동시에 prompt생성
* input_variables 인자를 사용해 변수를 지정한다
* 템플릿에 작성한 변수가 input_variables에 없으면 예외를 발생시켜준다

In [11]:
# template정의
template = "{language}는 누가 만들었나요"

prompt = PromptTemplate(
    template=template,
    input_variables=["language"],
)

print(prompt)

input_variables=['language'] input_types={} partial_variables={} template='{language}는 누가 만들었나요'


In [13]:
prompt.format(language="Python")

'Python는 누가 만들었나요'

** partial_variables **
* 연산중 미리 계산된 변수를 프롬프트 템플릿에 지정해 넣을수있다
* 항상 공통된 방식으로 가져오고 싶은  변수가 있는 경우 사용
ex)날짜, 시간


In [14]:
#템플릿 정의
template = "{language1}과  {language2}는 각각 누가 만들었나요?"

#prompttemplate 객체 생성
prompt =  PromptTemplate(
    template=template,
    input_variables=["language1"],
    partial_variables={
        "language2" : "Java" #dictionary형태로 partial_variables작성
    }
)

print(prompt)

input_variables=['language1'] input_types={} partial_variables={'language2': 'Java'} template='{language1}과  {language2}는 각각 누가 만들었나요?'


In [15]:
prompt.format(language1="Python")

'Python과  Java는 각각 누가 만들었나요?'

** partial()**
* 연산중에 미리 계산된 변수를 프롬프트 템플릿에 지정해서 넣을수있다

In [18]:
prompt_partial =  prompt.partial(language2 = "JavaScript")
print(prompt_partial)

input_variables=['language1'] input_types={} partial_variables={'language2': 'JavaScript'} template='{language1}과  {language2}는 각각 누가 만들었나요?'


In [19]:
prompt_partial.invoke("Python")

StringPromptValue(text='Python과  JavaScript는 각각 누가 만들었나요?')

In [20]:
chain = prompt_partial | llm

print(chain.invoke("Python").content)

Python은 귀도 반 로썸(Guido van Rossum)에 의해 1991년에 처음 개발되었습니다. 그는 이 언어를 만들 때 코드의 가독성과 사용의 용이성을 중시했습니다.

JavaScript는 브렌던 아이크(Brendan Eich)에 의해 1995년에 개발되었습니다. 그는 넷스케이프(Netscape)에서 일할 당시 웹 브라우저에서 동적인 기능을 추가하기 위해 JavaScript를 만들었습니다.

두 언어 모두 현재 널리 사용되고 있으며, 각각의 분야에서 중요한 역할을 하고 있습니다.


### 파일로부터 template읽어오기
* prompt를 편하게 작성 및 수정
* 상황에 맞게 파일을 작성해 두면 필요할때 마다 꺼내서 쓸수있다



In [22]:
from langchain_core.prompts import load_prompt

# prompt =  load_prompt("경로", encoding="utf-8")
prompt =  load_prompt("prompts/language_simple.yaml", encoding="utf-8")

print(prompt)


input_variables=['language'] input_types={} partial_variables={} template='{language}언어에 대해 3줄로 설명해줘'


In [23]:
prompt.format(language = "Python")

'Python언어에 대해 3줄로 설명해줘'

In [24]:
prompt2 = load_prompt("prompts/language.yaml", encoding="utf-8")

print(prompt2.format(language="Python"))

Python 언어에 대해 설명해주세요
언어의 특징을 다음의 양식에 맞게 정리하세요
300자 내외로 작성하세요
한글로 작성하세요
---
#양식
1. 특징
2. 제작자
3. 대표적인 프레임워크
4. 많이 사용되는 분야



In [26]:
from langchain_core.output_parsers import StrOutputParser

chain = prompt2 | ChatOpenAI(model_name ="gpt-4o-mini", temperature=0.0) | StrOutputParser()

answer = chain.invoke("Java")

print(answer)

1. 특징: Java는 객체 지향 프로그래밍 언어로, 플랫폼 독립성을 강조합니다. "Write Once, Run Anywhere"라는 슬로건 아래, JVM(Java Virtual Machine)을 통해 다양한 운영체제에서 실행 가능합니다. 강력한 메모리 관리와 예외 처리 기능을 제공하며, 풍부한 라이브러리와 API를 지원합니다.

2. 제작자: Java는 1995년 썬 마이크로시스템즈(Sun Microsystems)의 제임스 고슬링(James Gosling)에 의해 개발되었습니다.

3. 대표적인 프레임워크: Spring, Hibernate, JavaServer Faces(JSF), Apache Struts 등이 있습니다.

4. 많이 사용되는 분야: 웹 애플리케이션, 모바일 애플리케이션(Android), 기업용 소프트웨어, 빅데이터 처리 및 클라우드 컴퓨팅 등 다양한 분야에서 널리 사용됩니다.


### chatpromptTemplate
* 대화 목록을 프롬프트로 주입하고자 할떄 사용할수있다
* 메시지는 튜플형태로 전달
    * ("role","message") 구성되고, 리스트형태로 생성가능
    
**role**
* System :시스템 설정메세지로 주로 전역설정을 할때 사용
* human : 사용자의 입력메세지
* ai : ai의 답변 메시지

In [27]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_template("{language}의 제작자는 누구인가요")

print(chat_prompt)

input_variables=['language'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], input_types={}, partial_variables={}, template='{language}의 제작자는 누구인가요'), additional_kwargs={})]


In [28]:
chat_prompt.format(language="python")

'Human: python의 제작자는 누구인가요'

In [29]:
chat_template = ChatPromptTemplate.from_messages(
    [
        # role, message
        ("system", "당신은 친절한 인공지능 어시스턴트입니다 이름은{name}입니다"),
        ("human" ,"반가워요"),
        ("ai", "안녕하세요 무엇을 도와드릴까요"),
        ("human","{user_input}")
    ]
)

messages = chat_template.format_messages(
    name = "구찌", user_input = "당신의 이름은 무엇입니까"
    
)

print(messages)

[SystemMessage(content='당신은 친절한 인공지능 어시스턴트입니다 이름은구찌입니다', additional_kwargs={}, response_metadata={}), HumanMessage(content='반가워요', additional_kwargs={}, response_metadata={}), AIMessage(content='안녕하세요 무엇을 도와드릴까요', additional_kwargs={}, response_metadata={}), HumanMessage(content='당신의 이름은 무엇입니까', additional_kwargs={}, response_metadata={})]


In [30]:
print(llm.invoke(messages).content)

제 이름은 구찌입니다! 당신과 대화하게 되어 기쁩니다. 어떤 도움이 필요하신가요?


### messageplaceholder
* 아직 확정된 메시지가 아니지만, 나중에 채워질 메시지위치를 잡아두기 위해 사용한다
* 보통 대화 기록을 하고 싶을떄 사용한다

In [1]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

chat_prompt = ChatPromptTemplate(
    [
        (
            "system",
            "당신은 요약 전문ai어시스턴트입니다. 당신의 주요업무는 주요키워드로 대화를 요약하는것입니다"
        ),
            MessagesPlaceholder(variable_name="conversation"),
            ("human","지금까지의 대화를 {word_count}단어로 요약합니다")
    ]
)

print(chat_prompt)

input_variables=['conversation', 'word_count'] input_types={'conversation': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[langchain_core.m

In [34]:
formatted_chat_prompt = chat_prompt.format(
    word_count=5,
    conversation=[
        ("human", "파이썬에서 리스트에서 중복된 숫자를 제거하고 싶은데 어떻게 하면 될까?"),
        ("ai", "아주 간단해요! set()을 사용하면 중복을 쉽게 제거할 수 있답니다."),
        ("human", "set()이 뭔지는 잘 모르겠지만 엄청 간단하네! 이걸로 해결됐어 ㅎㅎ"),
        ("ai", "네, set은 중복을 허용하지 않는 자료형이에요! 정말 유용하죠. 앞으로도 파이썬의 편리한 기능들을 많이 알려드릴 수 있어요~ 😊")
    ]
)

print(formatted_chat_prompt)

System: 당신은 요약 전문ai어시스턴트입니다. 당신의 주요업무는 주요키워드로 대화를 요약하는것입니다
Human: 파이썬에서 리스트에서 중복된 숫자를 제거하고 싶은데 어떻게 하면 될까?
AI: 아주 간단해요! set()을 사용하면 중복을 쉽게 제거할 수 있답니다.
Human: set()이 뭔지는 잘 모르겠지만 엄청 간단하네! 이걸로 해결됐어 ㅎㅎ
AI: 네, set은 중복을 허용하지 않는 자료형이에요! 정말 유용하죠. 앞으로도 파이썬의 편리한 기능들을 많이 알려드릴 수 있어요~ 😊
Human: 지금까지의 대화를 5단어로 요약합니다


In [35]:
chain =  chat_prompt | llm | StrOutputParser()

In [3]:
input = {
    "word_count":5,
    "conversation" : [
        ("human", "파이썬에서 리스트에서 중복된 숫자를 제거하고 싶은데 어떻게 하면 될까?"),
        ("ai", "아주 간단해요! set()을 사용하면 중복을 쉽게 제거할 수 있답니다."),
        ("human", "set()이 뭔지는 잘 모르겠지만 엄청 간단하네! 이걸로 해결됐어 ㅎㅎ"),
        ("ai", "네, set은 중복을 허용하지 않는 자료형이에요! 정말 유용하죠. 앞으로도 파이썬의 편리한 기능들을 많이 알려드릴 수 있어요~ 😊")
    ]
}

# chain.invoke(input)

In [63]:
# input_data = {
#     "word_count": 5,
#     "conversation": [
#         {"role": "human", "content": "파이썬에서 리스트에서 중복된 숫자를 제거하고 싶은데 어떻게 하면 될까?"},
#         {"role": "ai", "content": "아주 간단해요! set()을 사용하면 중복을 쉽게 제거할 수 있답니다."},
#         {"role": "human", "content": "set()이 뭔지는 잘 모르겠지만 엄청 간단하네! 이걸로 해결됐어 ㅎㅎ"},
#         {"role": "ai", "content": "네, set은 중복을 허용하지 않는 자료형이에요! 정말 유용하죠. 앞으로도 파이썬의 편리한 기능들을 많이 알려드릴 수 있어요~ 😊"}
#     ]
# }

# result = chain.invoke(input_data)
# print(result)

OllamaEndpointNotFoundError: Ollama call failed with status code 404. Maybe your model is not found and you should pull the model with `ollama pull gemma:9b`.

In [37]:
!pip install langchain-community

   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---------------------------------------- 2.4/2.4 MB 34.2 MB/s eta 0:00:00


In [4]:
from langchain_community.llms import Ollama

#model
llm = Ollama(model="gemma2:9b")

response = llm.invoke("지구의 자전주기는?")

C:\Users\20109\AppData\Local\Temp\ipykernel_5308\1111038470.py:4: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="gemma2:9b")


In [3]:
from langchain.chat_models import ChatOllama

model = ChatOllama(
    model = "gemma:9b",
    temperature=0.1
)

C:\Users\20109\AppData\Local\Temp\ipykernel_5308\882105600.py:3: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  model = ChatOllama(


In [5]:
chain = chat_prompt | model | StrOutputParser("")

chain.invoke(input)

NameError: name 'chat_prompt' is not defined